**Introduction to AI - Project (Task 2)**

**Student Name:** Nguyễn Ngọc Thạch

**Student ID:** 2201700077



This AI model is built to process the **MNIST database** using PyTorch in Google Colab.

* **Description:** The MNIST (Modified National Institute of Standards and Technology) database is a collection of handwritten digits commonly used for training various image processing systems. It consists of 28x28 pixel ***grayscale*** images of ***handwritten digits from 0 to 9***.

* **Size:** The dataset contains a training set of 60,000 examples and a test set of 10,000 examples.

# Step 1: Import necessary libraries


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Step 2: Set up device (CPU or GPU)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


# Step 3: Define hyperparameters


In [ ]:
batch_size = 64
learning_rate = 0.01
num_epochs = 5

# Step 4: Load and preprocess the MNIST dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
'''
The mean and standard deviation values used in this case (0.1307 and 0.3081)
are precalculated statistics for the MNIST dataset. These values are commonly used for normalizing the MNIST dataset.
'''

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 31940702.51it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1080642.43it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9067476.66it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3791150.00it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Step 5: Define the neural network architecture


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

# Step 6: Instantiate the model, loss function, and optimizer


In [ ]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Step 7: Training loop

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Step 8: Testing loop

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

# Step 9: Main training and testing loop


In [ ]:
start_time = time.time()

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

end_time = time.time()
total_time = end_time - start_time

print(f"Total training time: {total_time:.2f} seconds")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303825
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.675387
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.679875
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.504711
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.245012
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.497394
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.637266
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.600801
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.510882
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.471456

Test set: Average loss: 0.0021, Accuracy: 9568/10000 (95.68%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.311235
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.274878
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.302508
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.477271
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.446788
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.508454
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.314421
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.254665
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.51706

# Step 10: Summary

In [ ]:
print(f"Device: {device}")
print(f"Model architecture:\n{model}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Device: cpu
Model architecture:
Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
Total parameters: 1199882
Trainable parameters: 1199882


# Conclusion

This model is a convolutional neural network (CNN) designed for processing the **MNIST database** of handwritten digits. It consists of two convolutional layers followed by dropout layers for regularization. The flattened features are passed through two fully connected layers for classification. The model has approximately 1.2 million parameters, all of which are trainable. With an **accuracy of 96.94%**, the model demonstrates strong performance in classifying handwritten digits from the MNIST dataset.